### Data Cleaning

Importing Libraries and Reading Data Files:

In [1]:
import pandas as pd
import numpy as np

In [2]:
wsb = pd.read_csv('./datasets/wsb_dirty.csv').drop(columns='Unnamed: 0')
stocks = pd.read_csv('./datasets/stocks_dirty.csv').drop(columns='Unnamed: 0')

In [3]:
wsb.shape

(9999, 4)

In [4]:
stocks.shape

(9998, 4)

Merging DataFrames:

In [5]:
df = pd.concat([wsb, stocks], axis=0, ignore_index=True)
df.head()

,subreddit,title,selftext,created_utc
0,wallstreetbets,"Made 45k on QQQ puts in 3 days, will finally p...",NaN,1642196072
1,wallstreetbets,There is a mining program giving unprecedented...,NaN,1642195904
2,wallstreetbets,TUESDAY = $ B B I G 🚀🚀🚀LFG 🚀🚀🚀,[removed],1642195841
3,wallstreetbets,PTN almost at the double Botham 🥳🍾💵💵💵💵💵💵,NaN,1642195792
4,wallstreetbets,I love weeklies. Fuck AMC,NaN,1642195724


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19997 entries, 0 to 19996
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subreddit    19997 non-null  object
 1   title        19997 non-null  object
 2   selftext     14169 non-null  object
 3   created_utc  19997 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 625.0+ KB


In [7]:
df.shape

(19997, 4)

Checking for nonsense and duplicates in 'title':

In [8]:
df.title.value_counts().head()

APPS (Digital Turbine) going down – much worse than a price correction                        11
Am I doing this right?                                                                         8
Everyone else is making money on Heliogen (HLGN) and it's time we did too (by shorting it)     8
New to stocks                                                                                  8
Number of Nasdaq Stocks Down 50% or More Is Almost at a Record                                 7
Name: title, dtype: int64

In [9]:
len(df.title.unique())

18940

Dropping duplicates from title:

In [10]:
df = df.drop_duplicates(subset='title', keep='first', ignore_index=True)

In [11]:
df.shape

(18940, 4)

Checking for nonsense and duplicates in 'selftext':

In [12]:
df.selftext.value_counts().head(3)

[removed]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [13]:
df.loc[df.selftext=='[removed]'].head(3)

,subreddit,title,selftext,created_utc
2,wallstreetbets,TUESDAY = $ B B I G 🚀🚀🚀LFG 🚀🚀🚀,[removed],1642195841
55,wallstreetbets,Is Elon Musk guilty of personal insider trading,[removed],1642191402
73,wallstreetbets,Pets to Rocket on Tuesday!,[removed],1642190118


In [14]:
df.loc[df.selftext=='[deleted]'].head(3)

,subreddit,title,selftext,created_utc
604,wallstreetbets,Diamonds hands are always good in the clutch,[deleted],1642127108
1872,wallstreetbets,When you time the market perfectly.,[deleted],1641996200
2104,wallstreetbets,Wasn’t a bad day today. Hodling for some more ...,[deleted],1641959287


In [15]:
df.loc[df.selftext.isna()].head(3)

,subreddit,title,selftext,created_utc
0,wallstreetbets,"Made 45k on QQQ puts in 3 days, will finally p...",NaN,1642196072
1,wallstreetbets,There is a mining program giving unprecedented...,NaN,1642195904
3,wallstreetbets,PTN almost at the double Botham 🥳🍾💵💵💵💵💵💵,NaN,1642195792


A large number of the posts have nan for selftext which appears to be related to the post only containing an image, video, or other embedded object; no text.

A massive portion of the posts have '[removed]' or '[deleted]' values.

Additionally, there are a fair amount of duplicate values in selftext.

However I think the titles of removed or deleted values can still be useful for analysis so I don't necessarrily want to drop all of them. I would like to remove duplicates but don't want to remove all rows containing selftext of 'deleted' or 'removed' in the process.

#### Removing all duplicates except removed, deleted, or nan from selftext:

In [16]:
#https://stackoverflow.com/questions/58269642/remove-drop-duplicates-except-where-row-contains-certain-string

df = df[~df.duplicated(['selftext']) | (df['selftext'] == '[removed]') | (df['selftext'] == '[deleted]') | (df['selftext'].isna())]


In [17]:
df.shape

(18791, 4)

Now besides removed and deleted, there are only 1 post of each selftext.

In [18]:
df.selftext.value_counts().head(3)

[removed]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18791 entries, 0 to 18939
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subreddit    18791 non-null  object
 1   title        18791 non-null  object
 2   selftext     13268 non-null  object
 3   created_utc  18791 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 734.0+ KB


### Creating all_text colummn in df

After removing all duplicate values from selftext and title, I want to join the two columns into a new 'all_text' column that ignores removed, deleted, or nan values in selftext.

In [20]:
#This is the subset of the dataframe for which I need to set all_text = df.title:
df[(df['selftext'] == '[removed]') | (df['selftext'] == '[deleted]') | (df['selftext'].isna())].head()


,subreddit,title,selftext,created_utc
0,wallstreetbets,"Made 45k on QQQ puts in 3 days, will finally p...",NaN,1642196072
1,wallstreetbets,There is a mining program giving unprecedented...,NaN,1642195904
2,wallstreetbets,TUESDAY = $ B B I G 🚀🚀🚀LFG 🚀🚀🚀,[removed],1642195841
3,wallstreetbets,PTN almost at the double Botham 🥳🍾💵💵💵💵💵💵,NaN,1642195792
4,wallstreetbets,I love weeklies. Fuck AMC,NaN,1642195724


In [27]:
#first I set all_text = the sum of title and selftext columns for the whole df:
df['all_text'] = df['title'] + ' ' + df['selftext']


In [28]:
#for selftext = nan,, all_text is = nan, to fix:
df['all_text'].loc[df['selftext'].isna()] = df['title']


In [29]:
#for selftext = '[removed]', all_text tags the '[removed]' string to the end of title, to fix:
df['all_text'].loc[df['selftext'] == '[removed]'] = df['title']


In [30]:
#for selftext = '[deleted]', all_text tags the '[deleted]' string to the end of title, to fix:
df['all_text'].loc[df['selftext'] == '[deleted]']  = df['title']


In [31]:
df.head(6)

,subreddit,title,selftext,created_utc,all_text
0,wallstreetbets,"Made 45k on QQQ puts in 3 days, will finally p...",NaN,1642196072,"Made 45k on QQQ puts in 3 days, will finally p..."
1,wallstreetbets,There is a mining program giving unprecedented...,NaN,1642195904,There is a mining program giving unprecedented...
2,wallstreetbets,TUESDAY = $ B B I G 🚀🚀🚀LFG 🚀🚀🚀,[removed],1642195841,TUESDAY = $ B B I G 🚀🚀🚀LFG 🚀🚀🚀
3,wallstreetbets,PTN almost at the double Botham 🥳🍾💵💵💵💵💵💵,NaN,1642195792,PTN almost at the double Botham 🥳🍾💵💵💵💵💵💵
4,wallstreetbets,I love weeklies. Fuck AMC,NaN,1642195724,I love weeklies. Fuck AMC
5,wallstreetbets,How will SPY close on Jan 17?,\n\n[View Poll](https://www.reddit.com/poll/s4...,1642195595,How will SPY close on Jan 17? \n\n[View Poll](...


#### Creating post_length column to look at longest and shortest posts by word count

In [40]:
df['post_length'] = [len(s.split()) for s in df['all_text']]
df.head()

,subreddit,title,selftext,created_utc,all_text,post_length
0,wallstreetbets,"Made 45k on QQQ puts in 3 days, will finally p...",NaN,1642196072,"Made 45k on QQQ puts in 3 days, will finally p...",16
1,wallstreetbets,There is a mining program giving unprecedented...,NaN,1642195904,There is a mining program giving unprecedented...,26
2,wallstreetbets,TUESDAY = $ B B I G 🚀🚀🚀LFG 🚀🚀🚀,[removed],1642195841,TUESDAY = $ B B I G 🚀🚀🚀LFG 🚀🚀🚀,9
3,wallstreetbets,PTN almost at the double Botham 🥳🍾💵💵💵💵💵💵,NaN,1642195792,PTN almost at the double Botham 🥳🍾💵💵💵💵💵💵,7
4,wallstreetbets,I love weeklies. Fuck AMC,NaN,1642195724,I love weeklies. Fuck AMC,5


In [42]:
df.sort_values(by='post_length').head(10)

,subreddit,title,selftext,created_utc,all_text,post_length
2730,wallstreetbets,Sad,NaN,1641883153,Sad,1
3810,wallstreetbets,🤪🤪🤪🤪,NaN,1641752111,🤪🤪🤪🤪,1
3809,wallstreetbets,ouch...,NaN,1641752173,ouch...,1
1491,wallstreetbets,Bullish,NaN,1642025046,Bullish,1
3804,wallstreetbets,This.,NaN,1641753064,This.,1
6673,wallstreetbets,Guh?!,NaN,1641416603,Guh?!,1
8321,wallstreetbets,MRVI,[removed],1641226629,MRVI,1
8039,wallstreetbets,👏👏👏👏,NaN,1641251654,👏👏👏👏,1
9389,wallstreetbets,SAFEMOON,[removed],1641019637,SAFEMOON,1
4801,wallstreetbets,😬🥴,NaN,1641586805,😬🥴,1


In [56]:
df.sort_values(by='post_length', ascending=False).head(10)

,subreddit,title,selftext,created_utc,all_text,post_length
10359,stocks,Wall Street Week Ahead for the trading week be...,Good Saturday afternoon to all of you here on ...,1641659209,Wall Street Week Ahead for the trading week be...,5902
11553,stocks,Wall Street Week Ahead for the trading week be...,Good Saturday afternoon to all of you here on ...,1641059082,Wall Street Week Ahead for the trading week be...,5054
18507,stocks,Wall Street Week Ahead for the trading week be...,Good Saturday morning to all of you here on r/...,1636809411,Wall Street Week Ahead for the trading week be...,4667
18372,stocks,Why I believe $MAPS WM Holdings is undervalued,WM Holdings is in my opinion is misunderstood ...,1636903112,Why I believe $MAPS WM Holdings is undervalued...,3968
15407,stocks,"TDOC holder, trying to calm myself...",**Trying to wrap my mind around this: About a...,1638455211,"TDOC holder, trying to calm myself... **Tryin...",3643
12347,stocks,Wall Street Week Ahead for the trading week be...,Good Sunday afternoon to all of you here on r/...,1640540466,Wall Street Week Ahead for the trading week be...,3608
9077,wallstreetbets,Automotive Roundup 2021 Part 2: 4 new SPACS fo...,Reposting due to over officious bot\n\n# 4 New...,1641086257,Automotive Roundup 2021 Part 2: 4 new SPACS fo...,3391
14713,stocks,"Nassim Taleb Fooled by Randomness, The Black S...",Nassim Taleb**\n\n# **Fooled by Randomness**\n...,1638857654,"Nassim Taleb Fooled by Randomness, The Black S...",3206
16323,stocks,Wall Street Week Ahead for the trading week be...,Good Friday afternoon to all of you here on r/...,1637961249,Wall Street Week Ahead for the trading week be...,3046
13245,stocks,Wall Street Week Ahead for the trading week be...,Good Saturday morning to all of you here on r/...,1639828068,Wall Street Week Ahead for the trading week be...,3029


#### Exporting df to csv for text processing and EDA

In [62]:
df.to_csv('./datasets/data.csv')

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18791 entries, 0 to 18939
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subreddit    18791 non-null  object
 1   title        18791 non-null  object
 2   selftext     13268 non-null  object
 3   created_utc  18791 non-null  int64 
 4   all_text     18791 non-null  object
 5   post_length  18791 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.5+ MB
